# Selection Criteria strategy.

This notebook is used to calculate the FOM: signal/$\sqrt {bkg}$.

bkg: background

In this case, signal is get from MC events and background is get from data events.

In [1]:
import time
import os
import numba

from coffea.nanoevents import BaseSchema
 
import awkward as ak
import numpy as np
from coffea import processor, hist

from coffea.nanoevents.methods import candidate
ak.behavior.update(candidate.behavior)

import mplhep
import matplotlib.pyplot as plt
plt.style.use(mplhep.style.CMS)

In [2]:
muon_cols = ['Muon_charge', 'Muon_dxy', 'Muon_dxyErr', 'Muon_dz', 'Muon_dzErr', 'Muon_eta', 'Muon_isGlobal', 'Muon_mass',
             'Muon_phi', 'Muon_pt', 'Muon_ptErr', 'Muon_softId', 'Muon_vtxIdx', 'Muon_vtxFlag', 'Muon_simIdx']

dimu_cols = ['Dimu_pt', 'Dimu_eta', 'Dimu_phi', 'Dimu_rap', 'Dimu_mass', 'Dimu_charge', 'Dimu_vtxIdx', 'Dimu_chi2', 'Dimu_dl',
             'Dimu_dlErr', 'Dimu_dlSig', 'Dimu_cosphi', 'Dimu_x', 'Dimu_y', 'Dimu_z', 'Dimu_t1muIdx', 'Dimu_t2muIdx',]

gen_part_cols = ['GenPart_eta', 'GenPart_genPartIdxMother', 'GenPart_mass', 'GenPart_pdgId', "GenPart_phi", "GenPart_pt", 'GenPart_status',
                'GenPart_Id', 'GenPart_parpdgId', 'GenPart_sparpdgId', 'GenPart_numberOfDaughters', 'GenPart_nstchgdaug', 'GenPart_vx', 
                'GenPart_vy', 'GenPart_vz', 'GenPart_mvx', 'GenPart_mvy', 'GenPart_mvz', 'GenPart_recIdx']

dstar_cols = ['Dstar_pt', 'Dstar_eta', 'Dstar_phi', 'Dstar_rap', 'Dstar_deltam', 'Dstar_deltamr', 'Dstar_vtxIdx', 'Dstar_hasMuon',
              'Dstar_D0pt', 'Dstar_D0eta', 'Dstar_D0phi', 'Dstar_D0mass', 'Dstar_D0chi2', 'Dstar_D0dl', 'Dstar_D0dlErr',
              'Dstar_D0dlSig', 'Dstar_D0cosphi', 'Dstar_D0x', 'Dstar_D0y', 'Dstar_D0z',
              'Dstar_Kpt', 'Dstar_Keta', 'Dstar_Kphi', 'Dstar_KvtxIdx', 'Dstar_Kchindof', 'Dstar_KnValid', 'Dstar_KnPix', 'Dstar_Kdxy',
              'Dstar_Kdz', 'Dstar_Kchg',
              'Dstar_pipt', 'Dstar_pieta', 'Dstar_piphi', 'Dstar_pivtxIdx', 'Dstar_pichindof', 'Dstar_pinValid', 'Dstar_pinPix',
              'Dstar_pidxy', 'Dstar_pidz', 'Dstar_pichg',
              'Dstar_pispt', 'Dstar_piseta', 'Dstar_pisphi', 'Dstar_pisvtxIdx', 'Dstar_pischindof', 'Dstar_pisnValid', 'Dstar_pisnPix',
              'Dstar_pisdxy', 'Dstar_pisdz', 'Dstar_simIdx']

D0_PDG_MASS = 1.864

def get_vars_dict(events, col_list):
    dict = {}
    col = ''
    for c in col_list:
        if c.startswith('Muon'):
            col = c[5:]
        elif c.startswith('Dimu'):
            col = c[4:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('D0'):
            col = c[2:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('Dstar'):
            col = c[5:]
            if col.startswith('_'): col = col[1:]
        elif c.startswith('PVtx'):
            col = c[5:]
        elif c.startswith("GenPart"):
            col = c[8:]
        else:
            Exception('Not good!')

        if col == 'x' or col == 'y' or col == 'z':
            col = 'vtx_' + col

        if len(events[c]) == 0:
            dict[col] = np.array([])
        else:
            dict[col] = events[c]
    return dict

In [3]:
import re

files_data = []
with os.scandir("/eos/user/m/mabarros/Data17UL/CharmoniumRun2017F_AOD/0000/") as it:
    for file in it:
        if file.name.endswith('.root') and (file.stat().st_size != 0):
            files_data.append(file.path)

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
            
files_data.sort(key=natural_keys)

In [4]:
def create_plot1d(hist1d, log=False, density=False, ax=None):
    from matplotlib.offsetbox import AnchoredOffsetbox, TextArea
    plt.style.use(mplhep.style.CMS)
    plt.rcParams.update({
        'font.size': 16,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    })
    fill_opts = {
    'alpha': 0.8,
    'edgecolor':(0,0,0,.5)
    }
    
    ax = hist.plot1d(hist1d, ax=ax, fill_opts=fill_opts, density=density)
    
    if log:
        ax.set_yscale('log')
        ax.set_ylim(1, None)
    else:
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3), useMathText=True)
    
    axis = hist1d.axes()[0]
    centers = axis.centers()
    values = np.where(hist1d.values().get(()) < 0, 0, hist1d.values().get(()))
    
    # compute mean and std:
    mean = np.sum(values*centers)/np.sum(values)
    std = np.sqrt(np.sum(values*((centers - mean)**2))/np.sum(values))
    
    annotation = TextArea(f"Total: {np.sum(values):.2e}" \
                    + "\n" + f"Mean: {mean:.2e}" \
                    + "\n" + f"Std: {std:.2e}", textprops=dict(size=14))
    
    at = AnchoredOffsetbox('upper right', child=annotation)
    at.patch.set_facecolor('None')
    #ax.add_artist(at)
    
    ax.legend().remove()
    
    return ax

In [5]:
def association(cand1, cand2):
    ''' Function for association of the particles. The cuts that operates on all of them and 
    computation of quantities can go here. individual cuts can go on the main processing'''

    #cut_dstar_back = ((cand2.deltamr > 0.143) & (cand2.deltamr < 0.148))
    #cand2 = cand2[cut_dstar_back]

    asso = ak.cartesian([cand1, cand2])    

    asso = asso[asso.slot0.vtxIdx == asso.slot1.vtxIdx]
    #asso = asso[ak.num(asso) > 0]
    cand1 = ak.zip({
            'pt': asso.slot0.pt,
            'eta': asso.slot0.eta,
            'phi': asso.slot0.phi,
            'mass': asso.slot0.mass,
            'charge': asso.slot0.charge}, with_name="PtEtaPhiMCandidate")

    cand2 = ak.zip({
            'pt': asso.slot1.pt,
            'eta': asso.slot1.eta,
            'phi': asso.slot1.phi,
            'mass': asso.slot1.mass,
            'charge': asso.slot1.charge}, with_name="PtEtaPhiMCandidate")

    asso['deltarap'] = asso.slot0.rap - asso.slot1.rap
    asso['cand'] = cand1 + cand2
    
    return asso

In [6]:
class DataTestProcessor(processor.ProcessorABC):
    def __init__(self):
        self._accumulator = processor.dict_accumulator({
            'JpsiDstar_deltarap' : hist.Hist("Events", hist.Bin("deltarap", "$\Delta y$", 50, -5, 5)),
            'JpsiDstar_mass' : hist.Hist("Events", hist.Bin("mass", "$m_{J/\psi D*}$ [GeV]", 100, 0, 100)),
            
            
        })
        

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        output = self.accumulator.identity()
        
        # test if there is any events in the file
        if len(events) == 0:
            return output
        
        # Collection extraction
        Dimus = ak.zip({**get_vars_dict(events, dimu_cols)}, with_name="PtEtaPhiMCandidate")
        Muons = ak.zip({**get_vars_dict(events, muon_cols)}, with_name="PtEtaPhiMCandidate")
        Dstars = ak.zip({'mass': (events.Dstar_D0mass + events.Dstar_deltamr),
                        'charge': events.Dstar_pischg,
                        **get_vars_dict(events, dstar_cols)}, 
                        with_name="PtEtaPhiMCandidate")        
        
        ## Rec dimuon cuts
        
        #Dimu cuts charge = 0, mass cuts and chi2...
        Dimu = Dimus[Dimus.charge == 0]

        Dimu = Dimu[(Dimu.mass > 2.95) & (Dimu.mass < 3.25)]
       
        # Get the Muons from Dimu, for cuts in their params
        Muon = ak.zip({'0': Muons[Dimu.t1muIdx], '1': Muons[Dimu.t2muIdx]})
        
        ## Rec muon cuts
        
        # Muon softId cuts
        soft_id = (Muon.slot0.softId > 0) & (Muon.slot1.softId > 0)
        Dimu = Dimu[soft_id]
        Muon = Muon[soft_id]

        # Muon pt and eta cuts
        muon_pt_cut = (Muon.slot0.pt > 3) & (Muon.slot1.pt > 3)
        Dimu = Dimu[muon_pt_cut]
        Muon = Muon[muon_pt_cut]
        
        # Cuts for Dstar
        Dstar = Dstars[~Dstars.hasMuon]

        Dstar = Dstar[(Dstar.Kpt > 0.5) & (Dstar.pipt > 0.5)]

        Dstar = Dstar[(Dstar.Kchindof < 2.5) & (Dstar.pichindof < 2.5)]

        Dstar = Dstar[(Dstar.KnValid > 4) & (Dstar.pinValid > 4) & (Dstar.KnPix > 1) & (Dstar.pinPix > 1)]

        Dstar = Dstar[(Dstar.Kdxy < 0.1) & (Dstar.pidxy < 0.1)]

        Dstar = Dstar[(Dstar.Kdz < 1) & (Dstar.pidz < 1)]

        # pis cuts
        Dstar = Dstar[Dstar.pispt > 0.3]

        Dstar = Dstar[Dstar.pischindof < 3]

        Dstar = Dstar[Dstar.pisnValid > 2]

        # D0 of Dstar cuts
        Dstar = Dstar[Dstar.D0cosphi > 0.99]

        Dstar = Dstar[(Dstar.D0mass < D0_PDG_MASS + 0.025) & (Dstar.D0mass > D0_PDG_MASS - 0.025)]
        
        Dstar = Dstar[Dstar.D0pt > 8]

        Dstar = Dstar[Dstar.D0dlSig > 3]

        Dstar['wrg_chg'] = (Dstar.Kchg == Dstar.pichg)
        
        Dstar = Dstar[~Dstar.wrg_chg]
      
        DimuDstar = association(Dimu, Dstar)
        #DimuDstar = DimuDstar[ak.num(DimuDstar.slot1) > 0]
        
        # To save accumulator
        
        DimuDstar_acc = processor.dict_accumulator({})
        DimuDstar_acc['Dimu'] = processor.dict_accumulator({})
        DimuDstar_acc['Dstar'] = processor.dict_accumulator({})
        
        
        for var in DimuDstar.fields:
            if (var == '0') or (var == '1'):
                continue
            elif var == 'cand':
                # loop over candidates fields (x, y, z, t and charge)
                for i0 in DimuDstar[var].fields:
                    # Create an entry on the dict accumulator with a field.
                    DimuDstar_acc[i0] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar[var][i0])))
            # If it is neither 0 and 1 or cand, it should be delta rapidity
            else:
                DimuDstar_acc[var] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar[var])))
        
        # loop for Jpsi from JpsiDstar
        for var in DimuDstar.slot0.fields:
            DimuDstar_acc['Dimu'][var] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar.slot0[var])))
            
        # loop for Dstar from JpsiDstar
        for var in DimuDstar.slot1.fields:
            DimuDstar_acc['Dstar'][var] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar.slot1[var])))
        
        # saves DimuDstar_acc in the output
        output['DimuDstar'] = DimuDstar_acc
            
        '''DimuDstar_acc['Dimu'] = processor.dict_accumulator({})
        DimuDstar_acc['Dstar'] = processor.dict_accumulator({})
        for var in DimuDstar.fields:
            if (var == '0') or (var =='1'):
                continue
            elif var == 'cand':
                for i0 in DimuDstar[var].fields:
                    DimuDstar_acc[i0] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar[var][i0])))
            else:
                DimuDstar_acc[var] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar[var])))

        for var in DimuDstar.slot0.fields:
            DimuDstar_acc['Dimu'][var] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar.slot0[var])))

        for var in DimuDstar.slot1.fields:
            DimuDstar_acc['Dstar'][var] = processor.column_accumulator(ak.to_numpy(ak.flatten(DimuDstar.slot1[var])))
        DimuDstar_acc['nDimuDstar'] = processor.column_accumulator(ak.to_numpy(ak.num(DimuDstar)))
        output['DimuDstar'] = DimuDstar_acc'''
        
        
        
        '''if len(DimuDstar) > 0:
            for i0 in range(len(DimuDstar)):
                print(i0)
                for i1 in range(len(DimuDstar[i0])):
                    print(f"Reco Dimu: ({DimuDstar.slot0.slot0.pt[i0][i1]}, {DimuDstar.slot0.slot0.eta[i0][i1]}, {DimuDstar.slot0.slot0.phi[i0][i1]})")
                    print(f" Gen Dimu: ({DimuDstar.slot0.slot1.pt[i0][i1]}, {DimuDstar.slot0.slot1.eta[i0][i1]}, {DimuDstar.slot0.slot1.phi[i0][i1]})")
                    print(f"Reco Dstar: ({DimuDstar.slot1.slot0.pt[i0][i1]}, {DimuDstar.slot1.slot0.eta[i0][i1]}, {DimuDstar.slot1.slot0.phi[i0][i1]})")
                    print(f" Gen Dstar: ({DimuDstar.slot1.slot1.pt[i0][i1]}, {DimuDstar.slot1.slot1.eta[i0][i1]}, {DimuDstar.slot1.slot1.phi[i0][i1]})")
                    print(f"Reco deltarap: {DimuDstar.deltarap[i0][i1]}, Gen deltarap: {DimuDstar.gen_deltarap[i0][i1]}")
                    print("-----------------------------------------------------------------")'''
       
        return output

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
data = {"test": files_data[:200]}

tstart = time.time()

output = processor.run_uproot_job(data,
                                    treename='Events',
                                    processor_instance=DataTestProcessor(),
                                    executor=processor.futures_executor, #iterative, futures
                                    executor_args={"schema": BaseSchema, 'workers': 8, 'skipbadfiles': True},
                                    chunksize=360000,
                                    )

print(f"Process finished in: {time.time() - tstart:.2f} s")

Preprocessing:   0%|          | 0/200 [00:00<?, ?file/s]

Processing:   0%|          | 0/195 [00:00<?, ?chunk/s]

Process finished in: 1509.23 s


In [23]:
##################### Association - J/Psi and D* #####################

import uproot3

# Dstar
all_asso_dstar = output['DimuDstar']['Dstar']['deltamr'].value
wrg_chg = output['DimuDstar']['Dstar']['wrg_chg'].value
dstar_wrong_charge = output['DimuDstar']['Dstar']['deltamr'].value[wrg_chg]
dstar_right_charge = output['DimuDstar']['Dstar']['deltamr'].value[~wrg_chg]

cut = (dstar_right_charge >= 0.14) & (dstar_right_charge <= 0.16)
dstar_right_charge = dstar_right_charge[cut]

# Jpsi
jpsi = output['DimuDstar']['Dimu']['mass'].value
jpsi = jpsi[cut]

root_file = 'DstarJpsi_asso_charmonium2017B.root'

with uproot3.recreate(root_file) as ds:
    ds['JpsiDstar'] = uproot3.newtree({"dstar_mass": "float32", "jpsi_mass": "float32"})
    ds['JpsiDstar'].extend({"dstar_mass": dstar_right_charge, "jpsi_mass": jpsi})